In [233]:
import numpy as np
from numpy.random import default_rng
import pandas as pd
import csv

In [234]:
doc_list = []
doc1 = "You may shoot me with your words,"
doc2 = "You may cut me with your eyes,"
doc3 = "Leaving behind nights of terror and fear I rise"
doc4 = "Pe-un picior de plai, pe-o gura de rai"
doc_list = [doc1, doc2, doc3, doc4]
num_docs = len(doc_list)
doc = "".join(doc_list).lower().replace(",", " ").replace("-", " ")
print(doc)

you may shoot me with your words you may cut me with your eyes leaving behind nights of terror and fear i risepe un picior de plai  pe o gura de rai


In [235]:
k = 4
shingles = set()
shingles = {doc[i:i+k] for i in range(len(doc) - k + 1)}

shingles_list = list(shingles)

d_prime = len(shingles_list)
shingles_index = [0] * d_prime
for idx in range(1, d_prime + 1):
    shingles_index[idx-1] = idx

print(d_prime)
print(shingles_list)
print(shingles_index)

126
['cut ', 'lai ', ' de ', ' un ', 'pe o', 'eavi', 'e wi', 'ear ', ' me ', 's le', ' wit', 'es l', 'or a', 'ar i', 'r an', 'and ', 'you ', ' lea', ' ris', 'rise', 'ith ', 'epe ', 'ts o', 'nd f', 'ror ', ' i r', 'd ni', 'ay c', ' pe ', 'd fe', 'g be', 'cior', 'r i ', 'gura', ' o g', 'hts ', ' you', 'of t', 'ehin', 'r ey', 'ura ', 'our ', 'leav', '  pe', ' nig', 'e ra', 'yes ', 'e o ', 'hind', 'avin', 'i  p', 'plai', ' may', 'isep', 'fear', 'e pl', ' cut', 's yo', 'ut m', 'terr', ' sho', 'rror', 'ay s', ' pla', 'ghts', 'de p', 'word', 'o gu', ' eye', 'ords', ' wor', 'ving', 'r de', 'r wo', 'with', 'y sh', ' fea', 'ot m', 't me', 'nigh', 'pe u', 'behi', 's of', 'shoo', ' and', ' rai', 'ai  ', 'de r', 'th y', 'ou m', 'un p', ' of ', 'e un', 'pici', 'ing ', 'ind ', 'sepe', 'erro', ' pic', 'nd n', 'or d', 'i ri', 'f te', 'h yo', 'ra d', 'your', 'may ', 'icio', 'me w', 'ng b', 'eyes', 'n pi', 'ur w', 'y cu', ' ter', 'ur e', 'ds y', 'rds ', 'u ma', 'ight', 'a de', 'hoot', 'oot ', 'ior ', ' g

In [236]:
out_path = 'shingles_table.csv'
with open(out_path, 'w', newline = '', encoding = 'utf-8') as f:
    writer = csv.writer(f)
    writer.writerow([len(shingles_list)])
    for x, idx in zip(shingles_list, shingles_index):
        writer.writerow([x, idx])

In [237]:
size = d_prime
c = [[0 for _ in range(size)] for _ in range(num_docs)]
for i, doc in enumerate(doc_list):
    doc = doc.lower().replace(",", " ").replace("-", " ")
    for j, s in zip(shingles_index, shingles_list):
        if s in doc:
            c[i][j-1] = 1

def get_jaccard(v1, v2):
    x = np.logical_and(v1, v2).sum()
    y = np.logical_or(v1, v2).sum()
    return x / y
    
c = np.array(c, dtype = int)
J = [[0 for _ in range(num_docs)] for _ in range(num_docs)]
for i in range(num_docs):
    for j in range(num_docs):
        J[i][j] = jaccard(c[i], c[j])
J = np.array(J, dtype = float)
print(J)

[[1.    0.425 0.    0.   ]
 [0.425 1.    0.    0.   ]
 [0.    0.    1.    0.   ]
 [0.    0.    0.    1.   ]]


In [238]:
def get_minhash_signatures(c, L, seed = 0):
    num_docs, size = c.shape
    rng = default_rng(seed)
    perms = [rng.permutation(np.arange(1, size + 1)) for _ in range(L)]
    M = np.zeros((num_docs, L), dtype = int)
    for i in range(num_docs):
        for j in range(L):
            for k, perm_idx in enumerate(perms[j]):
                if c[i, perm_idx-1] == 1:
                    M[i, j] = k
                    break
                else:
                    M[i, j] = size
    return M

min_hash1 = get_minhash_signatures(c, L = 4, seed = 2427348)
min_hash2 = get_minhash_signatures(c, L = 10, seed = 2427348)

print('L = 4')
for i in range(num_docs):
    print(f"x{i+1} =", min_hash1[i].tolist())
print('L = 10')
for i in range(num_docs):
    print(f"x{i+1} =", min_hash2[i].tolist())

L = 4
x1 = [0, 4, 2, 0]
x2 = [0, 2, 1, 15]
x3 = [8, 0, 7, 2]
x4 = [5, 3, 8, 1]
L = 10
x1 = [0, 4, 2, 0, 1, 4, 18, 4, 1, 0]
x2 = [0, 2, 1, 15, 0, 1, 0, 3, 8, 0]
x3 = [8, 0, 7, 2, 2, 2, 1, 1, 4, 1]
x4 = [5, 3, 8, 1, 4, 0, 4, 0, 0, 4]


In [239]:
def get_jaccard_from_signature(M):
    J = np.zeros((num_docs, num_docs), dtype = float)
    for i in range(num_docs):
        for j in range(num_docs):
            J[i, j] = np.mean(M[i] == M[j])
    return J

J1 = get_jaccard_from_signature(min_hash1)
J2 = get_jaccard_from_signature(min_hash2)
print('L = 4')
print(J1)
print('L = 10')
print(J2)

L = 4
[[1.   0.25 0.   0.  ]
 [0.25 1.   0.   0.  ]
 [0.   0.   1.   0.  ]
 [0.   0.   0.   1.  ]]
L = 10
[[1.  0.2 0.  0. ]
 [0.2 1.  0.  0. ]
 [0.  0.  1.  0. ]
 [0.  0.  0.  1. ]]
